# 다중 벡터저장소 검색기(MultiVectorRetriever)

하나의 문서에 대해 다수의 벡터(임베딩)를 생성하고, 이들을 이용해 더 정밀하고 유연한 검색을 가능하게 하는 리트리버입니다.
특히 문서가 여러 주제나 문맥을 포함하고 있을 때, 이를 다각도로 임베딩하여 검색 정확도를 높이는 데 유리


[ 장점 ]
1. 문서의 다면적 의미를 반영
문서 하나에 여러 개의 의미 단위가 있을 때, 각 의미에 해당하는 벡터를 생성.
예: 하나의 기술 백서가 AI, 보안, 네트워크를 모두 다룬다면, 각각의 임베딩을 따로 생성.
장점: 쿼리가 문서 내 특정 측면에 관련되어 있어도 검색 가능.

2. 장문 문서에 유리
전체 문서를 단일 임베딩으로 요약하면 정보 손실이 큼.
여러 벡터를 만들면 긴 문서의 다양한 주제를 모두 포착 가능.
장점: 보고서, 논문, 정책 문서 등의 검색 정밀도 증가.

3. 의미 기반의 정밀 검색 가능
다양한 쿼리 표현을 더 잘 포착할 수 있음.
하나의 쿼리가 문서의 일부 개념에만 맞아도 매칭 가능.
장점: 쿼리 다양성과 정밀도를 동시에 만족.

4. 질문 생성 기반으로 확장 가능
LLM을 이용해 문서에서 쿼리 예시(questions)를 뽑고, 이들을 임베딩 벡터로 사용.
즉, **"이 문서를 찾게 만들 수 있는 질문은 무엇일까?"**를 기반으로 벡터 생성.
장점: 질의응답(RAG) 기반 시스템과 잘 어울림.

[ 단점 ]
1. 벡터 저장 비용 증가
문서 1개당 벡터가 3개~10개 이상 생기므로, 전체 벡터 수가 급증.
→ 벡터DB 저장 비용 및 검색 성능 저하 가능성.
단점: 대규모 문서 코퍼스에서는 비용과 리소스 관리가 부담될 수 있음.

2. 임베딩 생성 비용 증가
벡터 생성 시 LLM을 이용해 쿼리/문맥을 추출하는 경우, 토큰 비용과 속도 부담이 큼.
단점: 초기 구축 비용이 높음.

3. 중복 문서 매칭 가능성
하나의 문서에서 여러 벡터가 생성되면, 하나의 쿼리에 대해 같은 문서가 중복으로 랭킹될 수 있음.
단점: 후처리로 중복 제거가 필요함.

4. 구현이 비교적 복잡
단일 임베딩 리트리버보다 파이프라인 구성이나 운영 난이도가 있음.
문서 → 질문 생성 → 질문 임베딩 → 문서 연결 등의 과정 필요.
단점: 기본적인 RAG 셋업보다 구현 부담이 있음.



LangChain에서는 문서를 다양한 상황에서 효율적으로 쿼리할 수 있는 특별한 기능, 바로 `MultiVectorRetriever`를 제공합니다. 이 기능을 사용하면 문서를 여러 벡터로 저장하고 관리할 수 있어, 정보 검색의 정확도와 효율성을 대폭 향상시킬 수 있습니다. 

`MultiVectorRetriever`를 활용해 문서당 여러 벡터를 생성하는 몇 가지 방법을 살펴보겠습니다.

**문서당 여러 벡터 생성 방법 소개**

1. **작은 청크 생성**: 문서를 더 작은 단위로 나눈 후, 각 청크에 대해 별도의 임베딩을 생성합니다. 이 방식을 사용하면 문서의 특정 부분에 좀 더 세심한 주의를 기울일 수 있습니다. 이 과정은 `ParentDocumentRetriever`를 통해 구현할 수 있어, 세부 정보에 대한 탐색이 용이해집니다.

2. **요약 임베딩**: 각 문서의 요약을 생성하고, 이 요약으로부터 임베딩을 만듭니다. 이 요약 임베딩은 문서의 핵심 내용을 신속하게 파악하는 데 큰 도움이 됩니다. 문서 전체를 분석하는 대신 핵심적인 요약 부분만을 활용하여 효율성을 극대화할 수 있습니다.

3. **가설 질문 활용**: 각 문서에 대해 적합한 가설 질문을 만들고, 이 질문에 기반한 임베딩을 생성합니다. 특정 주제나 내용에 대해 깊이 있는 탐색을 원할 때 이 방법이 유용합니다. 가설 질문은 문서의 내용을 다양한 관점에서 접근하게 해주며, 더 광범위한 이해를 가능하게 합니다.

4. **수동 추가 방식**: 사용자가 문서 검색 시 고려해야 할 특정 질문이나 쿼리를 직접 추가할 수 있습니다. 이 방법을 통해 사용자는 검색 과정에서 보다 세밀한 제어를 할 수 있으며, 자신의 요구 사항에 맞춘 맞춤형 검색이 가능해집니다.

텍스트 파일에서 데이터를 로드하고, 로드된 문서들을 지정된 크기로 분할하는 전처리 과정을 수행합니다.

분할된 문서들은 추후 벡터화 및 검색 등의 작업에 사용될 수 있습니다.


In [ ]:
from dotenv import load_dotenv
from langchain_community.document_loaders import PyMuPDFLoader #PDF 로더를 임포트 : PDF 파일을 읽고 페이지별로 문서(Document) 객체로 변환

loader = PyMuPDFLoader("data/gd.pdf") # ces.pdf을 로드할 수 있는 로더 인스턴스를 생성
docs = loader.load()                   # PDF를 로드하여 각 페이지를 LangChain의 Document 객체로 나누어 리스트로 반환

full_text = "\n\n".join([doc.page_content for doc in docs])
print(full_text)

1 
복리후생운영기준 
 
 
 
 
 
 
개정이력 
개정번호 
제∙개정일자 
개정내용 
비 고 
0 
2018.10.01 
복리후생 운영기준 제정 
제 정 
1 
2019.03.01 
출산 지원 확대 
개 정 
2 
2019.04.01 
해외 출장급지 조정 
개 정 
3 
2020.01.01 
장기근속 포상 변경 / 잔업교통비 변경 /  
국내, 해외출장비 변경 / 복지포인트 제도 신설 
개 정 
4 
2020.04.01 
출장 시 잔업교통비 미 지급 명문화, 
잔업 시 석식 지원 기준 명문화, 
22시 이후 퇴근 시 교통비 지원 명문화 
개 정 
5 
2020.08.01 
교통비 지원 결제수단 변경내용 개정 
개 정 
6 
2022.07.01 
잔업교통비 지원 개정 
사내 교육 강사료 지원 신설 
개 정 
7 
2024.02.01 
장기근속 포상 지급 내용 개정 
자격증 수당 지급 기준 개정 
개 정 
 
 
 
 
 
 
굿어스데이터 주식회사

2 
복리후생운영기준 
1. 경조금 및 경조휴가                                                (금액 단위 : 만원) 
구 분 
경조금 
휴가일수 
화환/조화 
결혼 
본인 
자녀 
본인 또는 배우자의 형제자매 
40 
20 
10 
7 
1 
1 
 
 
- 
수연 
본인 또는 배우자의 부모 
15 
1 
- 
출산 
배우자(관계 법령에 따름) 
10 
10 
 
사망 
본인 
배우자 
본인 또는 배우자의 부모 
자녀 
본인 또는 배우자의 조부모/외조부모 
본인 또는 배우자의 형제자매 
본인직계 백숙부모 
형제자매 또는 자녀의 배우자 
100 
40 
30 
30 
10 
10 
10 
10 
- 
7 
7 
7 
3 
3 
1 
1 
 
 
 
 
 
 
- 
- 
※ 경조휴가 일수는 역일 상의 발생일을 기준으로 하며, 경조휴가 기간 중의 휴일은 휴가일수에 산
입함(단, 배우자 출산휴가 중 휴일은 휴가일수 산입 제외) 
※ 경조금은 발생당일로부터 3개월 이

## Chunk + 원본 문서 검색

대용량 정보를 검색하는 경우, 더 작은 단위로 정보를 임베딩하는 것이 유용할 수 있습니다.

`MultiVectorRetriever` 를 통해 문서를 여러 벡터로 저장하고 관리할 수 있습니다.

`docstore` 에 원본 문서를 저장하고, `vectorstore` 에 임베딩된 문서를 저장합니다. 

이로써 문서를 더 작은 단위로 나누어 더 정확한 검색이 가능해집니다. 때에 따라서는 원본 문서의 내용을 조회할 수 있습니다.

vectorstore는 `찾는 역할`, docstore는 `보여주는 역할`입니다.

In [48]:
# 자식 청크를 인덱싱하는 데 사용할 벡터 저장소
import uuid                                                               # 고유한 문서 ID 생성을 위한 uuid 모듈 임포트
from langchain.storage import InMemoryStore                               # 원본 문서를 저장할 In-Memory 저장소 불러오기 (docstore 역할)
from langchain_chroma import Chroma                                       # 벡터 저장소로 사용할 Chroma 불러오기
from langchain_openai import OpenAIEmbeddings                             # 문서 임베딩 생성을 위한 OpenAI 임베딩 모델
from langchain_text_splitters import RecursiveCharacterTextSplitter       # 문서를 작은 조각으로 나누기 위한 텍스트 분할기
from langchain.retrievers.multi_vector import MultiVectorRetriever        # 여러 벡터(자식 조각)를 부모 문서와 연결해 검색하는 Retriever

vectorstore = Chroma(                                                     # 벡터 저장소 생성 (문서 조각을 임베딩하여 저장할 곳)
    collection_name="small_bigger_chunks",                                # 컬렉션 이름
    embedding_function=OpenAIEmbeddings(model="text-embedding-3-small"),  # 임베딩 모델 지정
)

store = InMemoryStore()        # 부모 문서를 저장할 In-Memory 저장소 생성 (docstore)

id_key = "doc_id"              # 벡터와 문서를 연결할 ID 키 이름 (문서 간 연결 시 사용)


# MultiVectorRetriever 생성
# → 여러 자식 조각을 벡터 저장소에서 검색하고, 해당 조각이 속한 부모 문서를 byte_store(docstore)에서 반환

retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    byte_store=store,
    id_key=id_key,
)

# 문서마다 고유한 UUID를 생성
doc_ids = [str(uuid.uuid4()) for _ in docs]

# 생성된 문서 ID 리스트 확인
doc_ids

['ee8788be-b32f-476b-981b-ff089a2f1e5e',
 'bbdc0f43-8e52-4c8c-ab28-484c5cb44f4d',
 '3a2954e8-28b2-41c3-916b-5025a765a4b2',
 'a158f240-e1be-4c54-ab48-0d6caea3fd12',
 'ba484b01-4585-4109-9d42-c7a1aa6663a3',
 '17b67d53-a519-4632-9b79-45eb192bd5a6',
 '05da9956-9596-4076-b70c-eaf823110eb9',
 'd9f4f15a-f747-4bb9-8507-f2a685d562a0',
 'b80b5b53-325c-41b8-93af-3e47d1dfa451',
 'ae4d32b1-a689-49bc-b1ea-84e213de3f72',
 '90037e0a-3dbe-4bbe-8e7c-dc458f191c8e',
 '6866110c-af24-4218-b408-5a22e71efdf4',
 'ec76cebd-f207-422f-84de-0527e32a9605']

여기서 큰 청크로 분할하기 위한 `parent_text_splitter`

더 작은 청크로 분할하기 위한 `child_text_splitter` 를 정의합니다.


In [49]:
# RecursiveCharacterTextSplitter 객체를 생성합니다.
parent_text_splitter = RecursiveCharacterTextSplitter(chunk_size=600)

# 더 작은 청크를 생성하는 데 사용할 분할기
child_text_splitter = RecursiveCharacterTextSplitter(chunk_size=200)

더 큰 Chunk인 Parent 문서를 생성합니다.

In [50]:
parent_docs = []

for i, doc in enumerate(docs):
    # 현재 문서의 ID를 가져옵니다.
    _id = doc_ids[i]
    # 현재 문서를 하위 문서로 분할
    parent_doc = parent_text_splitter.split_documents([doc])  # chunk_size=600

    for _doc in parent_doc:
        # metadata에 문서 ID 를 저장
        _doc.metadata[id_key] = _id
    parent_docs.extend(parent_doc)

`parent_docs` 에 기입된 `doc_id` 를 확인합니다.

In [51]:
# 생성된 Parent 문서의 메타데이터를 확인합니다.
#parent_docs[0].metadata
print(parent_docs[5])

page_content='◼ 학자금 지원금액 한도 : 인당 총 500만원/年 한도 내(단, 자녀 수는 제한 없음) 
◼ 유치원 
① 지원 금액 : 월 5만원 
② 지원 기간 : 취학 전 1년간 
③ 대상 학교 : 관인 및 비관인 유치원 
  ◼ 고등학교 : 실비 지급 (입학금 + 수업료 + 육성회비) 
  ◼ 전문대, 일반 대학교 : 실비 지급 (입학금 + 수업료 + 육성회비) 
    - 특수학교 (예능계, 외국인학교 등), 해외유학자녀는 국내 일반학생 처우 수준 (지급 총액)에 
      준함. 단, 이공계 실습비는 제외 
◼ 지급시기 
① 유 치 원 : 매월 지급 
② 고등학교 : 매 분기 말 기준 지급 
③ 전문, 일반대 : 매 반기 말 기준 지급' metadata={'producer': 'Microsoft® Word 2019', 'creator': 'Microsoft® Word 2019', 'creationdate': '2024-03-19T09:42:11+09:00', 'source': 'data/gd.pdf', 'file_path': 'data/gd.pdf', 'total_pages': 13, 'format': 'PDF 1.7', 'title': '인재전쟁(THE WAR FOR THE TALENT) 요약', 'author': 'stone', 'subject': '', 'keywords': '', 'moddate': '2024-03-19T09:42:11+09:00', 'trapped': '', 'modDate': "D:20240319094211+09'00'", 'creationDate': "D:20240319094211+09'00'", 'page': 2, 'doc_id': '3a2954e8-28b2-41c3-916b-5025a765a4b2'}


상대적으로 더 작은 Chunk인 Child 문서를 생성합니다.

In [53]:
child_docs = []
for i, doc in enumerate(docs):
    # 현재 문서의 ID를 가져옵니다.
    _id = doc_ids[i]
    # 현재 문서를 하위 문서로 분할
    child_doc = child_text_splitter.split_documents([doc]) # chunk_size=200
    for _doc in child_doc:
        # metadata에 문서 ID 를 저장
        _doc.metadata[id_key] = _id
    child_docs.extend(child_doc)

`child_docs` 에 기입된 `doc_id` 를 확인합니다.

In [54]:
# 생성된 Child 문서의 메타데이터를 확인합니다.
#child_docs[0].metadata

print(child_docs[5])

page_content='제 정 
1 
2019.03.01 
출산 지원 확대 
개 정 
2 
2019.04.01 
해외 출장급지 조정 
개 정 
3 
2020.01.01 
장기근속 포상 변경 / 잔업교통비 변경 /  
국내, 해외출장비 변경 / 복지포인트 제도 신설 
개 정 
4 
2020.04.01 
출장 시 잔업교통비 미 지급 명문화, 
잔업 시 석식 지원 기준 명문화,' metadata={'producer': 'Microsoft® Word 2019', 'creator': 'Microsoft® Word 2019', 'creationdate': '2024-03-19T09:42:11+09:00', 'source': 'data/gd.pdf', 'file_path': 'data/gd.pdf', 'total_pages': 13, 'format': 'PDF 1.7', 'title': '인재전쟁(THE WAR FOR THE TALENT) 요약', 'author': 'stone', 'subject': '', 'keywords': '', 'moddate': '2024-03-19T09:42:11+09:00', 'trapped': '', 'modDate': "D:20240319094211+09'00'", 'creationDate': "D:20240319094211+09'00'", 'page': 0, 'doc_id': 'ee8788be-b32f-476b-981b-ff089a2f1e5e'}


각각 분할된 청크의 수를 확인합니다.


In [55]:
print(f"분할된 parent_docs의 개수: {len(parent_docs)}")
print(f"분할된 child_docs의 개수: {len(child_docs)}")

분할된 parent_docs의 개수: 34
분할된 child_docs의 개수: 347


벡터저장소에 새롭게 생성한 작게 쪼개진 하위문서 집합을 추가합니다.

다음으로는 상위 문서는 생성한 UUID 와 맵핑하여 `docstore` 에 추가합니다.

- `mset()` 메서드를 통해 문서 ID와 문서 내용을 key-value 쌍으로 문서 저장소에 저장합니다.

In [56]:
# 벡터 저장소에 parent + child 문서를 추가
#    => 둘 다 검색 대상이 되며, 동일한 doc_id를 공유함
retriever.vectorstore.add_documents(parent_docs)
retriever.vectorstore.add_documents(child_docs)

# docstore 에 원본 문서를 저장
#    - doc_ids: 각 문서에 대한 고유 ID
#    - docs: 원본 전체 문서 목록
#    - zip(doc_ids, docs): (ID, 문서) 쌍으로 묶음
#    - mset(): 여러 개의 (key, value) 쌍을 한 번에 저장하는 함수

retriever.docstore.mset(list(zip(doc_ids, docs)))

유사도 검색을 수행합니다. 가장 유사도가 높은 첫 번째 문서 조각을 출력합니다.

여기서 `retriever.vectorstore.similarity_search` 메서드는 child + parent 문서 chunk 내에서 검색을 수행합니다.

In [67]:
# vectorstore의 유사도 검색을 수행합니다.
relevant_chunks = retriever.vectorstore.similarity_search(
    "출장 여비 계산 기준은?"
)
print(f"검색된 문서의 개수: {len(relevant_chunks)}")

검색된 문서의 개수: 4


In [68]:
for chunk in relevant_chunks:
    print(chunk.page_content, end="\n\n")
    print(">" * 100, end="\n\n")

5 
복리후생운영기준 
    ② 출장 결과 보고 및 여비 정산 
       > 출장자는 귀임 후 3일 이내에 출장 결과 보고를 하여야 함 
       > 출장 결과 보고는 온라인 문서(국내 출장 보고서)로 보고하여야 함 
   ◼ 출장 여비의 범위(아래 출장비 기준표 참고) 
     - 출장 여비는 출장기간 중 발생하는 교통비, 일당, 숙박비로 구성, 
      1) 교통비 : 지역별 KTX 표준운임으로 산정(자차 이용 시 미지급/자차사용신청서 작성 必) 
      2) 일당 : 시내 교통비, 식비 및 제반 경비를 포함(자차 이용 시 일당의 60% 지급) 
      3) 숙박비 : 숙박을 위한 객실사용료를 말하며, 급에 따라 차등하지 않음(100% 정액 지급) 
      4) 교재비 / 물품 구입비용 등 기타 경비는 출장 업무와 관련해 발생하더라도 출장여비와 
         별도로 구분하여 계산 
   ◼ 출장 여비의 계산 기준 
     - 기본원칙 
      1) 출장 여비의 일당 및 숙박비는 출장 여비 지급 기준에 따른 정액으로 계산하며,

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

3) 숙박비 : 숙박을 위한 객실사용료를 말하며, 급에 따라 차등하지 않음(100% 정액 지급) 
      4) 교재비 / 물품 구입비용 등 기타 경비는 출장 업무와 관련해 발생하더라도 출장여비와 
         별도로 구분하여 계산 
   ◼ 출장 여비의 계산 기준 
     - 기본원칙

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

복리후생운영기준 
    ② 출장 결과 보고 및 여비 정산 
       > 출장자는 귀임 후 3일 이내에 출장 결과 보고를 하여야 함 
      

이번에는 `retriever.invoke()` 메서드를 사용하여 쿼리를 실행합니다.

`retriever.invoke()` 메서드는 원본 문서의 전체 내용을 검색합니다.

In [70]:
relevant_docs = retriever.invoke("출장 여비 계산 기준은?")
print(f"검색된 문서의 개수: {len(relevant_docs)}", end="\n\n")
print("=" * 100, end="\n\n")
print(relevant_docs[0].page_content)

검색된 문서의 개수: 1


5 
복리후생운영기준 
    ② 출장 결과 보고 및 여비 정산 
       > 출장자는 귀임 후 3일 이내에 출장 결과 보고를 하여야 함 
       > 출장 결과 보고는 온라인 문서(국내 출장 보고서)로 보고하여야 함 
   ◼ 출장 여비의 범위(아래 출장비 기준표 참고) 
     - 출장 여비는 출장기간 중 발생하는 교통비, 일당, 숙박비로 구성, 
      1) 교통비 : 지역별 KTX 표준운임으로 산정(자차 이용 시 미지급/자차사용신청서 작성 必) 
      2) 일당 : 시내 교통비, 식비 및 제반 경비를 포함(자차 이용 시 일당의 60% 지급) 
      3) 숙박비 : 숙박을 위한 객실사용료를 말하며, 급에 따라 차등하지 않음(100% 정액 지급) 
      4) 교재비 / 물품 구입비용 등 기타 경비는 출장 업무와 관련해 발생하더라도 출장여비와 
         별도로 구분하여 계산 
   ◼ 출장 여비의 계산 기준 
     - 기본원칙 
      1) 출장 여비의 일당 및 숙박비는 출장 여비 지급 기준에 따른 정액으로 계산하며, 
         교통비는 출장비 기준표의 기준 운임에 따름(미 기재 지역은 인근지역 기준으로 지급) 
      2) 일당은 출장지내 이동시 시내 교통비, 식대 및 제반 잡비를 말하는 것으로 출장 일수에 
         따라 계산하고, 출장의 시점은 오전 0시로 하며 1일 미만의 단수는 1일로 계산 
         (예 : 12월 1일 19:00 출발, 12월 2일 09:00 귀임의 경우 2일임) 
      3) 숙박료는 숙박을 위한 객실사용료를 말하는 것으로 숙박 일수에 따라 계산하고,  
         숙박 일수의 계산은 오전 0시를 지남으로써 이를 1박으로 함 
         (예 : 12월 1일 19:00 출발, 12월 2일 09:00 귀임의 경우 1박임) 
      4) 교통비는 지역별 지급 기준에 의하여 정액으로 지급되며, 대중교통 이용을 원칙으로 함

리트리버(retriever)가 벡터 데이터베이스에서 기본적으로 수행하는 검색 유형은 유사도 검색입니다.

LangChain Vector Stores는 [Max Marginal Relevance](https://api.python.langchain.com/en/latest/vectorstores/langchain_core.vectorstores.VectorStore.html#langchain_core.vectorstores.VectorStore.max_marginal_relevance_search)를 통한 검색도 지원하므로, 이를 대신 사용하고 싶다면 다음과 같이 `search_type` 속성을 설정하면 됩니다.


- `retriever` 객체의 `search_type` 속성을 `SearchType.mmr`로 설정합니다.
  - 이는 검색 시 MMR(Maximal Marginal Relevance) 알고리즘을 사용하도록 지정하는 것입니다.


In [76]:
from langchain.retrievers.multi_vector import SearchType

# 검색 유형을 MMR (Maximal Marginal Relevance)로 설정
#  - 관련성 있는 결과 중 중복을 줄이고 다양한 정보를 포함하도록 유도
# 중복을 줄이면서도 관련성 높은 문서를 반환하도록 함
retriever.search_type = SearchType.mmr

# 관련 문서 전체를 검색
print(retriever.invoke("출장 여비 계산 기준은?")[0].page_content)

5 
복리후생운영기준 
    ② 출장 결과 보고 및 여비 정산 
       > 출장자는 귀임 후 3일 이내에 출장 결과 보고를 하여야 함 
       > 출장 결과 보고는 온라인 문서(국내 출장 보고서)로 보고하여야 함 
   ◼ 출장 여비의 범위(아래 출장비 기준표 참고) 
     - 출장 여비는 출장기간 중 발생하는 교통비, 일당, 숙박비로 구성, 
      1) 교통비 : 지역별 KTX 표준운임으로 산정(자차 이용 시 미지급/자차사용신청서 작성 必) 
      2) 일당 : 시내 교통비, 식비 및 제반 경비를 포함(자차 이용 시 일당의 60% 지급) 
      3) 숙박비 : 숙박을 위한 객실사용료를 말하며, 급에 따라 차등하지 않음(100% 정액 지급) 
      4) 교재비 / 물품 구입비용 등 기타 경비는 출장 업무와 관련해 발생하더라도 출장여비와 
         별도로 구분하여 계산 
   ◼ 출장 여비의 계산 기준 
     - 기본원칙 
      1) 출장 여비의 일당 및 숙박비는 출장 여비 지급 기준에 따른 정액으로 계산하며, 
         교통비는 출장비 기준표의 기준 운임에 따름(미 기재 지역은 인근지역 기준으로 지급) 
      2) 일당은 출장지내 이동시 시내 교통비, 식대 및 제반 잡비를 말하는 것으로 출장 일수에 
         따라 계산하고, 출장의 시점은 오전 0시로 하며 1일 미만의 단수는 1일로 계산 
         (예 : 12월 1일 19:00 출발, 12월 2일 09:00 귀임의 경우 2일임) 
      3) 숙박료는 숙박을 위한 객실사용료를 말하는 것으로 숙박 일수에 따라 계산하고,  
         숙박 일수의 계산은 오전 0시를 지남으로써 이를 1박으로 함 
         (예 : 12월 1일 19:00 출발, 12월 2일 09:00 귀임의 경우 1박임) 
      4) 교통비는 지역별 지급 기준에 의하여 정액으로 지급되며, 대중교통 이용을 원칙으로 함 
      5) 다만, 출

In [ ]:
from langchain.retrievers.multi_vector import SearchType

# 검색 유형을 similarity_score_threshold로 설정
#     - 문서와 쿼리 간의 임베딩 유사도 점수가 특정 임계값 이상일 때만 결과에 포함됩니다.
retriever.search_type = SearchType.similarity_score_threshold

#     - score_threshold = 0.1: 유사도 점수가 0.1 이상인 문서만 결과로 반환됩니다.
retriever.search_kwargs = {"score_threshold": 0.1}

# 관련 문서 전체를 검색
print(retriever.invoke("출장 여비 기준은?")[0].page_content)


5 
복리후생운영기준 
    ② 출장 결과 보고 및 여비 정산 
       > 출장자는 귀임 후 3일 이내에 출장 결과 보고를 하여야 함 
       > 출장 결과 보고는 온라인 문서(국내 출장 보고서)로 보고하여야 함 
   ◼ 출장 여비의 범위(아래 출장비 기준표 참고) 
     - 출장 여비는 출장기간 중 발생하는 교통비, 일당, 숙박비로 구성, 
      1) 교통비 : 지역별 KTX 표준운임으로 산정(자차 이용 시 미지급/자차사용신청서 작성 必) 
      2) 일당 : 시내 교통비, 식비 및 제반 경비를 포함(자차 이용 시 일당의 60% 지급) 
      3) 숙박비 : 숙박을 위한 객실사용료를 말하며, 급에 따라 차등하지 않음(100% 정액 지급) 
      4) 교재비 / 물품 구입비용 등 기타 경비는 출장 업무와 관련해 발생하더라도 출장여비와 
         별도로 구분하여 계산 
   ◼ 출장 여비의 계산 기준 
     - 기본원칙 
      1) 출장 여비의 일당 및 숙박비는 출장 여비 지급 기준에 따른 정액으로 계산하며, 
         교통비는 출장비 기준표의 기준 운임에 따름(미 기재 지역은 인근지역 기준으로 지급) 
      2) 일당은 출장지내 이동시 시내 교통비, 식대 및 제반 잡비를 말하는 것으로 출장 일수에 
         따라 계산하고, 출장의 시점은 오전 0시로 하며 1일 미만의 단수는 1일로 계산 
         (예 : 12월 1일 19:00 출발, 12월 2일 09:00 귀임의 경우 2일임) 
      3) 숙박료는 숙박을 위한 객실사용료를 말하는 것으로 숙박 일수에 따라 계산하고,  
         숙박 일수의 계산은 오전 0시를 지남으로써 이를 1박으로 함 
         (예 : 12월 1일 19:00 출발, 12월 2일 09:00 귀임의 경우 1박임) 
      4) 교통비는 지역별 지급 기준에 의하여 정액으로 지급되며, 대중교통 이용을 원칙으로 함 
      5) 다만, 출

In [ ]:
from langchain.retrievers.multi_vector import SearchType

# 검색 유형을 similarity(단순 유사도 기반 검색)로 설정하고,
# 한 번에 반환할 문서 개수를 1개(k=1)로 지정

retriever.search_type = SearchType.similarity
retriever.search_kwargs = {"k": 1}

# 관련 문서 전체를 검색
print(len(retriever.invoke("출장 여비 기준은?")))

1


## 요약본(summary)을 벡터저장소에 저장

요약은 종종 청크(chunk)의 내용을 보다 정확하게 추출할 수 있어 더 나은 검색 결과를 얻을 수 있습니다.

여기서는 요약을 생성하는 방법과 이를 임베딩하는 방법에 대해 설명합니다.


In [119]:
# PDF 파일을 로드하고 텍스트를 분할하기 위한 라이브러리 임포트
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# PDF 파일 로더 초기화
loader = PyMuPDFLoader("data/gd.pdf")

# 텍스트 분할
# 텍스트 분할기 생성: chunk_size=600 글자 단위로 분할하며, 분할 구간 사이에 50 글자 겹침을 둠
text_splitter = RecursiveCharacterTextSplitter(chunk_size=600, chunk_overlap=50)

# PDF 파일 로드 및 텍스트 분할 실행
split_docs = loader.load_and_split(text_splitter)

# 분할된 문서의 개수 출력
print(f"분할된 문서의 개수: {len(split_docs)}")

분할된 문서의 개수: 28


In [120]:
from langchain_core.documents import Document               # 문서 객체 정의용 임포트
from langchain_core.output_parsers import StrOutputParser   # 출력 결과를 문자열로 파싱하는 도구
from langchain_core.prompts import ChatPromptTemplate       # 대화형 프롬프트 템플릿 생성용
from langchain_openai import ChatOpenAI                     # OpenAI ChatGPT 모델 연동용


summary_chain = (
    {"doc": lambda x: x.page_content} # 입력 문서 객체에서 텍스트 내용(page_content)만 추출하여 "doc" 키에 매핑

    | ChatPromptTemplate.from_messages(  # 요약을 위한 ChatPromptTemplate 생성: 시스템 메시지와 사용자 메시지 포함
        [
            ("system", "당신은 한국어 문서 요약 전문가입니다."),
            (
                "user",
                "다음 문서를 세 문장으로 요약해 주세요. 요약은 글머리표 형식으로 작성해 주세요.\n\n{doc}",
            ),
        ]
    )
    # OpenAI의 ChatGPT 모델을 사용하여 요약 생성
    | ChatOpenAI(temperature=0, model="gpt-4o-mini") #창의력X
    | StrOutputParser()    # 모델 출력 결과를 문자열로 변환
)

In [121]:
# 문서 배치 처리
# chain.batch 메서드를 사용하여 docs 리스트의 문서들을 일괄 요약
summaries = summary_chain.batch(split_docs, {"max_concurrency": 10}) # 10개의 문서 동시 처리
len(summaries)

28

요약된 내용을 출력하여 결과를 확인합니다.


In [122]:
# 원본 문서의 내용을 출력합니다.
print(split_docs[17].page_content, end="\n\n")
# 요약을 출력합니다.
print("[요약]")
print(summaries[17])

> 일반 교통비를 지급 
④ 장기 출장의 기간 산정은 선행 일자순으로 차감하여 계산하며, 기타 예외 사항 발생이 
예견될 경우에는 주관 부서와 사전 협의를 해야 함 
⑤ 회사가 인정할 수 있는 지역 상황에 의하여 숙박비와 일당의 초과 지불이 불가피한 
경우는 별도의 품의에 의하여 그 실비를 인정 받을 수 있음 
   ◼ 기타사항 : 예외 사항 발생이 예견될 경우에는 주관 부서와 사전 협의 必 
 
9. 경조 출장비 
   ◼ 취지 
     - 임직원의 경조사 시 회사를 대표하여 경/조의를 표하기 위해 출장하는 경우 발생하는 
       경비를 보조함으로서 개인의 제반 경비부담을 최소화하기 위함 
   ◼ 지원기준 
     - 경조사 발생 건당 2명 이내(지원부서 제외) 
   ◼ 지급기준 
구 분 
숙박비 
교통비 
임 원 
실 비 
실 비 
직 원 
6만원 한도 내 실비

[요약]
- 일반 교통비는 지급되며, 장기 출장의 기간은 선행 일자순으로 차감하여 계산하고, 예외 사항 발생 시 주관 부서와 사전 협의가 필요하다.
- 회사가 인정하는 지역 상황에 따라 숙박비와 일당의 초과 지불이 불가피할 경우, 별도의 품의를 통해 실비를 인정받을 수 있다.
- 경조 출장비는 임직원의 경조사 시 발생하는 경비를 보조하며, 지원 기준은 경조사 건당 2명 이내로, 임원은 실비, 직원은 6만원 한도 내 실비로 지급된다.


`Chroma` 벡터 저장소를 초기화하여 자식 청크(child chunks)를 인덱싱합니다. 이때 `OpenAIEmbeddings`를 임베딩 함수로 사용합니다.

- 문서 ID를 나타내는 키로 `"doc_id"`를 사용합니다.


In [123]:
import uuid

# 요약 정보를 저장할 벡터 저장소를 생성합니다.
summary_vectorstore = Chroma(
    collection_name="summaries",
    embedding_function=OpenAIEmbeddings(model="text-embedding-3-small"),
)

# 부모 문서를 저장할 저장소를 생성합니다.
store = InMemoryStore()

# 문서 ID를 저장할 키 이름을 지정합니다.
id_key = "doc_id"

# 검색기를 초기화합니다. (시작 시 비어 있음)
retriever = MultiVectorRetriever(
    vectorstore=summary_vectorstore,  # 벡터 저장소
    byte_store=store,  # 바이트 저장소
    id_key=id_key,  # 문서 ID 키
)
# 문서 ID를 생성합니다.
doc_ids = [str(uuid.uuid4()) for _ in split_docs]

요약된 문서와 메타데이터(여기서는 생성한 요약본에 대한 `Document ID` 입니다)를 저장합니다.


In [124]:
summary_docs = [
    # summaries 리스트의 각 요약문 s에 대해 Document 객체 생성
    # page_content에는 요약된 텍스트 s를 넣고,
    # metadata에는 해당 요약이 어떤 원본 문서(doc_ids[i])에서 왔는지 문서 ID를 포함
    Document(page_content=s, metadata={id_key: doc_ids[i]})
    for i, s in enumerate(summaries) #  요약문(summaries)과 해당 문서 ID(doc_ids)를 인덱스로 맞춰 처리
]

요약본의 문서의 개수는 원본 문서의 개수와 일치합니다.


In [125]:
# 요약본의 문서의 개수
len(summary_docs)

28

- `retriever.vectorstore.add_documents(summary_docs)`를 통해 `summary_docs`를 벡터 저장소에 추가합니다.
- `retriever.docstore.mset(list(zip(doc_ids, docs)))`를 사용하여 `doc_ids`와 `docs`를 매핑하여 문서 저장소에 저장합니다.


In [126]:
retriever.vectorstore.add_documents(
    summary_docs
)  # 요약된 문서를 벡터 저장소에 추가합니다.

# 문서 ID와 문서를 매핑하여 문서 저장소에 저장합니다.
retriever.docstore.mset(list(zip(doc_ids, split_docs)))

`vectorstore` 객체의 `similarity_search` 메서드를 사용하여 유사도 검색을 수행합니다.


In [127]:
# 유사도 검색을 수행합니다.
result_docs = summary_vectorstore.similarity_search(
    "출장 여비 기준은?"
)

In [128]:
# 1개의 결과 문서를 출력합니다.
print(result_docs[0].page_content)

- 출장 기간이 연장된 것으로 판단되어 해당 기간 동안의 출장 여비가 지급된다.  
- 출장 여비 지급은 출장 기간의 연장에 따라 자동으로 이루어진다.  
- 이는 출장자의 경비 부담을 덜어주기 위한 조치이다.  


`retriever` 객체의 `invoke()` 사용하여 질문과 관련된 문서를 검색합니다.


In [129]:
# 관련된 문서를 검색하여 가져옵니다.
retrieved_docs = retriever.invoke("출장 여비 기준은?")
print(retrieved_docs[0].page_content)

출장 기간이 계속되는 것으로 간주하여 그 기간 동안의 출장 여비를 지급 함


## 가설 쿼리(Hypothetical Queries) 를 활용하여 문서 내용 탐색

LLM은 `특정 문서에 대해 가정할 수 있는 질문 목록을 생성`하는 데에도 사용될 수 있습니다.

이렇게 생성된 질문들은 임베딩(embedding)될 수 있으며, 이를 통해 문서의 내용을 더욱 깊이 있게 탐색하고 이해할 수 있습니다.

가정 질문 생성은 문서의 주요 주제와 개념을 파악하는 데 도움이 되며, 독자들이 문서 내용에 대해 더 많은 궁금증을 갖도록 유도할 수 있습니다.


아래는 `Function Calling` 을 활용하여 가설 질문을 생성하는 예제입니다.

In [131]:
functions = [
    {
        "name": "hypothetical_questions",  # 함수의 이름을 지정합니다.
        "description": "Generate hypothetical questions",  # 함수에 대한 설명을 작성합니다.
        "parameters": {  # 함수의 매개변수를 정의합니다.
            "type": "object",  # 매개변수의 타입을 객체로 지정합니다.
            "properties": {  # 객체의 속성을 정의합니다.
                "questions": {  # 'questions' 속성을 정의합니다.
                    "type": "array",  # 'questions'의 타입을 배열로 지정합니다.
                    "items": {
                        "type": "string"
                    },  # 배열의 요소 타입을 문자열로 지정합니다.
                },
            },
            "required": ["questions"],  # 필수 매개변수로 'questions'를 지정합니다.
        },
    }
]

`ChatPromptTemplate`을 사용하여 주어진 문서를 기반으로 3개의 가상 질문을 생성하는 프롬프트 템플릿을 정의합니다.

- `functions`와 `function_call`을 설정하여 가상 질문 생성 함수를 호출합니다.
- `JsonKeyOutputFunctionsParser`를 사용하여 생성된 가상 질문을 파싱하고, `questions` 키에 해당하는 값을 추출합니다.

In [132]:
from langchain_core.prompts import ChatPromptTemplate                               # 대화형 AI 프롬프트 템플릿을 생성하기 위한 모듈 임포트
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser  # OpenAI 함수 호출 응답에서 JSON 형태의 특정 키를 추출하는 출력 파서 임포트
from langchain_openai import ChatOpenAI                                             # OpenAI의 ChatGPT 모델과 연동하기 위한 클래스 임포트

hypothetical_query_chain = (
    {"doc": lambda x: x.page_content}
    # 아래 문서를 사용하여 답변할 수 있는 가상의 질문을 정확히 3개 생성하도록 요청합니다. 이 숫자는 조정될 수 있습니다.
    | ChatPromptTemplate.from_template(
        "아래 문서를 바탕으로 답변할 수 있는 가상의 질문 3개를 정확히 생성하세요. "
        "잠재적 사용자는 복리 후생 운영기준에 관심 있는 사람들입니다. 그들이 관심을 가질 만한 질문을 만들어 주세요.:\n\n{doc}"
    )
    | ChatOpenAI(max_retries=0, model="gpt-4o-mini").bind(   # OpenAI GPT-4o-mini 모델을 사용하여 질문 생성 (재시도 없이 max_retries=0 설정)
        functions=functions, function_call={"name": "hypothetical_questions"}    # 'functions'와 'function_call' 매개변수를 바인딩해 특정 함수 호출로 질문 생성 유도
    )
    # 모델 출력 결과에서 JSON의 "questions" 키에 해당하는 값만 추출하는 파서 적용
    | JsonKeyOutputFunctionsParser(key_name="questions")
)

문서에 대한 답변을 출력합니다.

- 출력은 생성한 3개의 가설 쿼리(Hypothetical Queries) 가 담겨 있습니다.


In [133]:
# 주어진 문서에 대해 체인을 실행합니다.
hypothetical_query_chain.invoke(split_docs[17])

['장기 출장 시 예외 사항이 발생했을 경우 어떻게 처리해야 하나요?',
 '경조사 시 출장비 지원 기준에 대해 더 알고 싶어요. 지원받을 수 있는 인원수와 금액은 어떻게 되나요?',
 '숙박비와 일당의 초과 지불이 필요한 경우, 어떤 절차를 통해 실비를 인정받을 수 있나요?']

`chain.batch` 메서드를 사용하여 `split_docs` 데이터에 대해 동시에 여러 개의 요청을 처리합니다.

In [135]:
# 문서 목록에 대해 가설 질문을 배치 생성
hypothetical_questions = hypothetical_query_chain.batch(
    split_docs, {"max_concurrency": 10}
)

In [136]:
hypothetical_questions[17]

['장기 출장이 필요한 경우, 사전 협의를 어떻게 진행해야 하나요?',
 '경조 출장비 지원 기준은 어떻게 되며, 지급받기 위해 어떤 서류가 필요하나요?',
 '회사에서 인정하는 지역 상황에 따른 숙박비 초과 지급 기준은 무엇인가요?']

아래는 이전에 진행했던 방식과 동일하게 생성한 가설 쿼리(Hypothetical Queries) 를 벡터저장소에 저장하는 과정입니다.


In [138]:
# 자식 청크를 인덱싱하는 데 사용할 벡터 저장소
hypothetical_vectorstore = Chroma(
    collection_name="hypo-questions", embedding_function=OpenAIEmbeddings()
)
# 부모 문서의 저장소 계층
store = InMemoryStore()

id_key = "doc_id"
# 검색기 (시작 시 비어 있음)
retriever = MultiVectorRetriever(
    vectorstore=hypothetical_vectorstore,
    byte_store=store,
    id_key=id_key,
)
doc_ids = [str(uuid.uuid4()) for _ in split_docs]  # 문서 ID 생성

`question_docs` 리스트에 메타데이터(문서 ID) 를 추가합니다.


In [140]:
question_docs = []
# hypothetical_questions 저장
for i, question_list in enumerate(hypothetical_questions):
    question_docs.extend(
        # 질문 리스트의 각 질문에 대해 Document 객체를 생성하고, 메타데이터에 해당 질문의 문서 ID를 포함시킵니다.
        [Document(page_content=s, metadata={id_key: doc_ids[i]}) for s in question_list]
    )

가설 쿼리를 문서에 추가하고, 원본 문서를 `docstore` 에 추가합니다.


In [142]:
# hypothetical_questions 문서를 벡터 저장소에 추가합니다.
retriever.vectorstore.add_documents(question_docs)

# 문서 ID와 문서를 매핑하여 문서 저장소에 저장합니다.
retriever.docstore.mset(list(zip(doc_ids, split_docs)))

`vectorstore` 객체의 `similarity_search` 메서드를 사용하여 유사도 검색을 수행합니다.


In [144]:
# 유사한 문서를 벡터 저장소에서 검색합니다.
result_docs = hypothetical_vectorstore.similarity_search(
    "출장 여비 기준은?"
)

아래는 유사도 검색 결과입니다.

여기서는 생성한 가설 쿼리만 추가해 놓은 상태이기 때문에, 생성한 가설 쿼리 중 유사도가 가장 높은 문서를 반환합니다.


In [145]:
# 유사도 검색 결과를 출력합니다.
for doc in result_docs:
    print(doc.page_content)
    print(doc.metadata)

출장 여비 발생 기준에 대한 세부 사항은 무엇인가요?
{'doc_id': 'a42c5a8d-b130-46ad-8457-788e9f028774'}
출장 여비 발생 기준에 대한 세부 사항은 무엇인가요?
{'doc_id': 'a42c5a8d-b130-46ad-8457-788e9f028774'}
출장 기간 동안 여비는 어떻게 산정되나요?
{'doc_id': '76998b44-cf02-4a5c-beed-45959137249c'}
출장 기간 동안 여비는 어떻게 산정되나요?
{'doc_id': '76998b44-cf02-4a5c-beed-45959137249c'}


`retriever` 객체의 `invoke` 메서드를 사용하여 쿼리와 관련된 문서를 검색합니다.


In [150]:
# 관련된 문서를 검색하여 가져옵니다.
retrieved_docs = retriever.invoke(result_docs[0].page_content)

# 검색된 문서를 출력합니다.
for doc in retrieved_docs:
    print(doc.page_content)

53,300 
107,000 
부산광역시 
서울 – 부산 
59,800 
120,000 
제주도(항공편) 
서울(김포) – 제주 
102,300 
205,000 
      ※ KTX 운임표 기준 / 강원도 춘천 : ITX 기준 / 제주도 : 대한항공 항공편 운임 기준 
      ※ 자차 사용 필요시, 반드시 자차사용신청서 상신(출장신청서에 자차 사용 및 사유 명시) 
      ※ 휴일 출장의 경우, 일당은 동일하지만 잔업교통비는 인정하지 않음 
   ◼ 출장 여비 발생 기준 
구 분 
교통비 
숙박비 
일 당 
업무 출장 
○ 
○ 
O 
교육 출장 
○ 
○ 
X 
경조 출장 
○ 
○ 
X
출장 기간이 계속되는 것으로 간주하여 그 기간 동안의 출장 여비를 지급 함
